# ChessBot Research Project Playground
here is where we can play against the bot (for now against ourself)

In [68]:
import random
import numpy as np
import chess
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Add, Activation
from keras.models import Model
from keras.models import load_model

In [59]:
class Node:
    def __init__(self, board, parent=None):
        self.board = board
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0
        self.untried_moves = list(self.board.legal_moves)

    def expand(self):
        move = random.choice(self.untried_moves)
        self.untried_moves.remove(move)
        new_board = self.board.copy()
        new_board.push(move)
        new_node = Node(new_board, self)
        self.children.append(new_node)
        return new_node

    def select_best_child(self):
        best_score = -float('inf')
        best_child = None
        for child in self.children:
            score = child.wins/child.visits + 0.5*np.sqrt(2*np.log(self.visits)/child.visits)
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

class MCTS:
    def __init__(self, simulations):
        self.simulations = simulations

    def play_random_game(self, board, model):
        node = Node(board)
        while not node.board.is_game_over():
            if node.untried_moves:
                move = random.choice(node.untried_moves)
                node = node.expand()
                if not node.board.is_legal(move):
                    continue
                node.board.push(move)
            else:
                move_evals = []
                for child in node.children:
                    child_board = child.board
                    move_evals.append(model.predict(child_board.fen())[0][0])
                best_move = node.children[np.argmax(move_evals)]
                node = best_move
        result = node.board.result()
        if result == "1-0":
            return 1
        elif result == "0-1":
            return -1
        else:
            return 0

    def search(self, board, model):
        root = Node(board)
        for i in range(self.simulations):
            node = root
            while node.untried_moves == [] and node.children != []:
                node = node.select_best_child()
            if node.untried_moves != []:
                child = node.expand()
                node = child
            result = self.play_random_game(node.board, model)
            while node != None:
                node.visits += 1
                if result == 1:
                    node.wins += 1
                node = node.parent
        if root.children:
            return max(root.children, key = lambda x: x.visits).board.pop()
        else:
            return None






In [69]:
def save_models(model1, model2, mcts1, mcts2):
    model1.save("model1.h5")
    model2.save("model2.h5")
    pickle.dump(mcts1, open("mcts1.pkl", "wb"))
    pickle.dump(mcts2, open("mcts2.pkl", "wb"))

def train_mcts(simulations_per_move, moves):
    # Create a new chess game
    game = chess.Board()
    # Create two MCTS objects
    mcts1 = MCTS(simulations_per_move)
    mcts2 = MCTS(simulations_per_move)

    i = 1
    while not game.is_game_over():
        # Make the best move according to the MCTS search
        move1 = mcts1.search(game, model1)
        print(f"Move {i}: {move1}")
        game.push(move1)
        move2 = mcts2.search(game, model2)
        print(f"Move {i+1}: {move2}")
        game.push(move2)
        i += 2
    # Get the result of the game
    result = game.result()
    print("Result:", result)
    # Update the losing model
    if result == "1-0":
        x_train = game.fen()
        y_train = model2.predict(x_train)
        model2.fit(x_train, y_train, verbose=0)
    elif result == "0-1":
        x_train = game.fen()
        y_train = model1.predict(x_train)
        model1.fit(x_train, y_train, verbose=0)
    # play a new game with the updated models
    game = chess.Board()
    save_models(model1, model2, mcts1, mcts2)
    return mcts1, mcts2

In [70]:
def create_model():
    input_layer = Input(shape=(8, 8, 1))
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    for i in range(9):
        y = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
        y = Conv2D(64, kernel_size=(3, 3), activation='linear', padding='same')(y)
        x = Add()([x, y])
        x = Activation('relu')(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(input_layer, x)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [72]:
# Create two copies of the model
model1 = create_model()
model2 = create_model()

# model1 = load_model('model1.h5')
# model2 = load_model('model2.h5')

# Set the number of simulations per move
simulations_per_move = 1000

# Set the number of moves to play
moves = 100

# Start the self-play training process
mcts1, mcts2 = train_mcts(simulations_per_move, moves)

KeyboardInterrupt: 